In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

mcc_freq = pd.read_csv("mcc_frequency.csv")
ttp_freq = pd.read_csv("trans_types_frequency.csv")
incomes  = pd.read_csv("Incomes.csv")
outcomes = pd.read_csv("Outcomes.csv")
outcomes_at_hours = pd.read_csv("/content/outcomes_at_hours.csv")
incomes_at_hours  = pd.read_csv("/content/Incomes_at_hour.csv")
genders  = pd.read_csv("genders.csv")
mcc_freq = mcc_freq.drop("Unnamed: 0", axis=1)
mcc_freq /= mcc_freq.sum().values
mcc_freq = mcc_freq.transpose()

def preprocess(data):
    data = data.set_index("Unnamed: 0").T
    scale = np.abs(data.sum().values)
    data = (data / scale).T
    return data
genders = preprocess(genders)
ttp_freq = preprocess(ttp_freq)
incomes  = preprocess(incomes )
outcomes = preprocess(outcomes)
incomes_at_hours = preprocess(incomes_at_hours)
outcomes_at_hours = preprocess(outcomes_at_hours)
df_all = pd.concat([ttp_freq,mcc_freq], axis =1)

X = df_all.fillna(0).to_numpy()
y = genders.fillna(0).to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

import xgboost as xgb
from sklearn.metrics import roc_auc_score

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 5,
    'learning_rate': 0.1,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight':  1

}

dtrain = xgb.DMatrix(X, label=y)
dtest = xgb.DMatrix(X_test, label=y_test)

model = xgb.train(params, dtrain, num_boost_round=100)


y_pred_proba = model.predict(dtest)

roc_auc = roc_auc_score(y_test, y_pred_proba)
model.save_model("model.json")
print("ROC-AUC:", roc_auc)



ROC-AUC: 0.944822595779195


In [ ]:
import xgboost as xgb
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 5,
    'learning_rate': 0.1,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight':  1

}
# Загрузка модели из файла
model = xgb.XGBClassifier(params,num_boost_round=100)
model.load_model("model.json")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:726: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


In [ ]:


# Predict using the XGBoost model
y_pred_proba = model.predict(X_test)

roc_auc_score(y_test, y_pred_proba)

0.7559810641602656

In [ ]:
y_pred_proba

array([0, 1, 0, ..., 0, 1, 0])

In [ ]:
y_train_df = pd.DataFrame(y_train,columns = ['target'])
y_test_df = pd.DataFrame(y_test,columns = ['target'])
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)

In [ ]:
df_train = pd.concat([X_train_df,y_train_df], axis =1)
df_test = pd.concat([X_test_df,y_test_df], axis =1)

In [ ]:
df_test

,0,1,2,3,4,5,6,7,8,9,...,330,331,332,333,334,335,336,337,338,target
0,0.0,0.000000,0.000000,0.000000,0.090909,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.408163,0.132653,0.000000,0.069388,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.247818,0.153578,0.010471,0.057592,0.013962,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.010471,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.052239,0.738806,0.000000,0.029851,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.015625,0.414062,0.000000,0.054688,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,0.0,0.183544,0.575949,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1508,0.0,0.203125,0.007812,0.000000,0.125000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1509,0.0,0.173469,0.193878,0.010204,0.244898,0.010204,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0
1510,0.0,0.134503,0.269006,0.000000,0.219298,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
outcomes_at_hours = pd.read_csv("/content/outcomes_at_hours.csv")

In [ ]:
outcomes_at_hours

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,c5f18a1d1d83fda62e51e6ae1d7b8911,-2679.39,-786.76,-268.73,-2465.98,-178.89,-1057.46,-1540.25,-6076.56,-18151.84,...,-68200.70,-44271.72,-71224.09,-38469.92,-99192.33,-41696.23,-64962.32,-65357.45,-32858.48,-9037.04
1,4b6861803556de90249e13d7cf66f0a9,-2176.99,-432.22,-73.22,-71.32,-218.78,-144.54,-21388.77,-1313.97,-1169.45,...,-17477.74,-39915.59,-42264.50,-25150.46,-10555.02,-27485.86,-4711.38,-1912.14,-2753.38,-5825.22
2,7d3f908e1cd1e3624a1445df5e04c4fa,-87714.78,0.00,-143.62,-504.97,-1230.01,-143.85,-22600.67,-19380.10,-48432.62,...,-19058.46,-19599.06,-7451.56,-15702.55,-1765.10,-3741.69,-396.72,0.00,0.00,-55757.04
3,40c585a809c1bd17dc6d283d61acd2d2,-2797.86,-42.57,0.00,0.00,0.00,0.00,0.00,0.00,-95524.95,...,-30873.93,-16255.60,-93810.56,-51095.25,-8294.15,-2176.83,-2735.60,-1134.02,0.00,0.00
4,5bd2ff9f6c591353577488b0dfca6dea,-22.86,0.00,-156.25,0.00,-148.46,-252.57,-1098.09,-8323.48,-32368.90,...,-8687.13,-12470.13,-11318.03,-14848.27,-12357.61,-13860.55,-3552.73,-2240.61,-71.73,-3348.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,596baadc284cb94a91c1fdd7a422a022,-28388.12,0.00,0.00,0.00,0.00,0.00,0.00,-10032.64,-14087.83,...,-78503.45,-54948.76,-59345.72,-25543.13,-8795.02,-3684.46,-8875.74,0.00,0.00,-37736.41
7556,9b317a4cff6bb3b630cd966f68db410b,-73.02,0.00,0.00,0.00,0.00,0.00,0.00,-72.29,-3415.46,...,-28292.08,-21213.88,-24497.34,-10421.60,-2872.74,-2170.41,-10415.97,-1127.19,0.00,0.00
7557,eb3b544e3c5293739f09dcd71a649b18,-86.87,-497.16,-13206.89,-34560.90,-32992.49,-15778.49,-32569.24,-27646.14,-7509.36,...,-774.53,-766.24,-287.62,0.00,0.00,0.00,0.00,0.00,0.00,-121.60
7558,e1c1a70bfcb000d1188745cef71b3b61,-16978.49,0.00,-145.29,0.00,0.00,0.00,0.00,-215.28,-722.37,...,-36393.21,-1841.72,-8703.96,-9420.67,-5739.61,-4112.78,-77135.58,-12052.77,-2535.60,-20891.16


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
xgb_config = {'use_gpu': True}
model = xgb.XGBClassifier(**xgb_config,n_estimators=500, learning_rate=0.05)
model.fit(X_train, y_train,
             early_stopping_rounds=5,
             eval_set=[(X_test, y_test)],
             verbose=False)
#y_pred = model.predict(X_test)
auc_score = roc_auc_score(y_test, y_pred)
print("ROC-AUC Score:", auc_score)

ROC-AUC Score: 0.7666470522750043


In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.02, 0.1, 1, 10,100]}

model = LogisticRegression()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)


grid_search.fit(X_train, y_train)

print("Наилучшие параметры:", grid_search.best_params_)
print("Лучшая модель:", grid_search.best_estimator_)


best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
auc_score = roc_auc_score(y_test, y_pred)
print("ROC-AUC Score:", auc_score)

Наилучшие параметры: {'C': 100}
Лучшая модель: LogisticRegression(C=100)
ROC-AUC Score: 0.7357888431219749


In [ ]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 5,
    'learning_rate': 0.1,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight':  1

}

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

model = xgb.train(params, dtrain, num_boost_round=100)


y_pred_proba = model.predict(dtest)

roc_auc = roc_auc_score(y_test, y_pred_proba)

print("ROC-AUC:", roc_auc)

ROC-AUC: 0.8551936611940747


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'min_child_weight': [1, 3, 5]
}

model = XGBClassifier()

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='roc_auc')


grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

y_pred_proba = grid_search.best_estimator_.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("ROC-AUC:", roc_auc)

In [ ]:
print("Best Parameters:", grid_search.best_params_)

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

train_pool = Pool(data=X_train, label=y_train)
test_pool = Pool(data=X_test, label=y_test)


params = {
    'iterations': [100],
    'eval_metric': ['AUC'],
    'random_seed': [42],
    'learning_rate': [0.01, 0.1],
    'depth': [4, 6, 8],
}


model = CatBoostClassifier()

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

y_pred_proba = grid_search.best_estimator_.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("ROC-AUC:", roc_auc)